In [35]:
import numpy as np
import pandas as pd

In [1]:
%reload_kedro
import networkx as nx
G = io.load('G_train')
edges_train = io.load('edges_train')
y_train = io.load('y_train')

2019-07-03 09:10:19,847 - INFO - ** Kedro project Twitter Network
2019-07-03 09:10:19,849 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/logging.yml
2019-07-03 09:10:19,853 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/catalog.yml
2019-07-03 09:10:19,860 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/credentials.yml
2019-07-03 09:10:19,863 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/parameters.yml
2019-07-03 09:10:19,870 - INFO - Defined global variables proj_dir, proj_name, conf and io
2019-07-03 09:10:19,872 - INFO - Loading data from `G_train` (PickleLocalDataSet)...
2019-07-03 09:11:17,820 - INFO - Loading data from `edges_train` (PickleLoc

In [6]:
DiG = io.load('DiG_train')

2019-07-03 09:12:30,574 - INFO - Loading data from `DiG_train` (PickleLocalDataSet)...


In [9]:
from twitter_network.nodes.features import resource_allocation, adamic_adar
import networkx.algorithms.link_prediction as lp

In [17]:
for i in range(1000):
    source, sink = edges_train[i]
    x1 = set(G.neighbors(source))
    x2 = set(G.neighbors(sink))
    if resource_allocation(x1, x2, DiG) > 0:
        break

In [24]:
source, sink = edges_train[4]
x1 = set(G.neighbors(source))
x2 = set(G.neighbors(sink))

print("Resource allocation:", resource_allocation(x1, x2, DiG))
print("Resource allocation (LP):", [x for u,v,x in lp.resource_allocation_index(G, [(source, sink)])])
print("Adamic adar:", resource_allocation(x1, x2, DiG))
print("Adamic adar (LP):", [x for u,v,x in lp.adamic_adar_index(G, [(source, sink)])])

Resource allocation: 0.018781128646348882
Resource allocation (LP): [0.01692800943677535]
Adamic adar: 0.018781128646348882
Adamic adar (LP): [0.7904076059278872]


In [22]:
x1.intersection(x2)

{'221443', '2753279', '2937523', '4130826', '848620'}

In [15]:
import pickle
with open("data/04_features/test_connectivity.pkl", 'rb') as f:
    connectivity = pickle.load(f)

In [22]:
connectivity.describe(percentiles = [k * 0.1 for k in range(10)])

count    2000.000000
mean       33.731500
std        51.559302
min         0.000000
0%          0.000000
10%         1.000000
20%         3.000000
30.0%       5.000000
40%         9.000000
50%        15.000000
60.0%      24.000000
70%        33.000000
80%        50.000000
90%        88.000000
max       435.000000
dtype: float64

In [21]:
from networkx.algorithms.approximation.connectivity import local_node_connectivity
local_node_connectivity(DiG, source, target, cutoff = 100)

15

In [38]:
# Assortativity
from networkx.algorithms.assortativity import *
print('Average neighbor degree: {}'.format(average_neighbor_degree(DiG, nodes = [source])[source]))

Average neighbor degree: 15856.105263157895


In [48]:
print('Dispersion: {}'.format(dispersion(G, source, target)))

Dispersion: 0.0


In [52]:
print('Triangles: {}'.format(triangles(G, source)))
print('Clustering: {}'.format(clustering(DiG, [source])[source]))
print('Average clustering: {}'.format(average_clustering(DiG, test)))

Triangles: 4438
Clustering: 0.03639042313335799
Average clustering: 0.18486187823334566


In [36]:
from twitter_network.nodes.features import link_dispersion
from networkx.algorithms.cluster import *
from sklearn.feature_selection import mutual_info_classif
N = 1000
disp = [link_dispersion(e, G) for e in edges_train[:N]]
source_clustering =  [clustering(G, [u])[u] for u, v in edges_train[:N]]
sink_clustering =  [clustering(G, [v])[v] for u, v in edges_train[:N]]
X = pd.DataFrame(dict(disp=disp,source_clustering = source_clustering, sink_clustering = sink_clustering))
X["avg_clustering"] = (X["source_clustering"]+X["sink_clustering"])/2
mutual_info_classif(X, y_train[:N])

array([0.02637773, 0.        , 0.27451182, 0.06727311])

In [37]:
from networkx.algorithms.structuralholes import *
constraint = [local_constraint(G, u, v) for u, v in edges_train[:5]]

KeyboardInterrupt: 

In [39]:
from networkx.algorithms.connectivity.connectivity import local_edge_connectivity
connectivity = [local_edge_connectivity(G, u, v) for u, v in edges_train[:5]]

KeyboardInterrupt: 